In [7]:
# Data conversion from CSV to Json 
import json 
import pandas as pd
f = open(r"data_web.json","r", encoding='utf8')

input = json.loads(f.read())
keys = list(input.keys())

features = list(input[keys[0]].keys())
features.remove("name")
new_data = {}
for key in ['id'] + features :
   new_data[key] = []

new_data['id'] = keys

for id in keys: 
   for f in features:
      new_data[f].append(input[id][f])

df = pd.DataFrame(new_data)
df.to_csv (r'Data/data_web.csv', index = None)

In [8]:
import pandas as pd
from surprise import *
from sklearn import preprocessing


In [42]:
def reform_dataset(df1): 
   # Merge subject/module names with feature names
   new_cols = []
   for i,j,k in zip(list(df1.iloc[3]), list(df1.iloc[2]),  list(df1.iloc[1])):
         new_cols.append(i + " :" + j + " (" + k + ")")

   p_cols = []
   #Keep only the percentage for each module
   for col in new_cols:
      if "**" in col or "%" in col :
         p_cols.append(col)

   # Create the new dataset
   new_df = df1
   new_df.columns = new_cols
   new_df = new_df.drop([0,1,2,3],axis=0)
   new_df = new_df[p_cols]
   return(new_df)

In [15]:
df = pd.read_csv(r'Data/data_web.csv')
df = df.dropna(subset=['id'])
df = df.drop(["total_time"],axis=1)

df["id"] =df["id"].astype('int64')

df.head()

,id,level_id,donuts,candy,level,progress,id Quizo,nbr Items,badge المثابرة,التركيز,...,Q_Worst_Time,worst_time_math,worst_time_science,worst_time_ar,worst_time_fr,Q_Avg_Time,Avg_Time_math,Avg_Time__science,Avg_Time_ar,Avg_Time_fr
0,1661,4.0,1.0,350.0,1.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,1677,3.0,20.0,350.0,1.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0,5,0,0
2,1681,3.0,10.0,350.0,1.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0,5,0,0
3,1683,3.0,2.0,350.0,1.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0
4,1700,3.0,46.0,350.0,1.0,0.0,NaN,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0,0,0,0,0


In [24]:
df.columns

Index(['id', 'level_id', 'donuts', 'candy', 'level', 'progress', 'id Quizo',
       'nbr Items', 'badge المثابرة', 'التركيز', 'النجاح', 'التميز',
       'الإرتقاء', 'total_Time_Education', 'nb_mistakes', 'correct Question',
       'correct_Q_Math', 'correct_Q_Science', 'correct_Q_Ar', 'correct_Q_Fr',
       'Q_Best_Time', 'best_time_math', 'best_time_science', 'best_time_ar',
       'best_time_fr', 'Q_Worst_Time', 'worst_time_math', 'worst_time_science',
       'worst_time_ar', 'worst_time_fr', 'Q_Avg_Time', 'Avg_Time_math',
       'Avg_Time__science', 'Avg_Time_ar', 'Avg_Time_fr'],
      dtype='object')

In [18]:
final_df = df.melt(id_vars="id")

final_df.columns = ["user", "item", "rating"]
final_df = final_df.fillna(0)
final_df

,user,item,rating
0,1661,level_id,4.0
1,1677,level_id,3.0
2,1681,level_id,3.0
3,1683,level_id,3.0
4,1700,level_id,3.0
...,...,...,...
571263,64664,Avg_Time_fr,0.0
571264,64670,Avg_Time_fr,0.0
571265,64673,Avg_Time_fr,0.0
571266,64677,Avg_Time_fr,0.0


In [22]:
normalized_arr = preprocessing.normalize([final_df['rating']])
final_df['rating'] = normalized_arr[0]
final_df

,user,item,rating
0,1661,level_id,9.842909e-07
1,1677,level_id,7.382182e-07
2,1681,level_id,7.382182e-07
3,1683,level_id,7.382182e-07
4,1700,level_id,7.382182e-07
...,...,...,...
571263,64664,Avg_Time_fr,0.000000e+00
571264,64670,Avg_Time_fr,0.000000e+00
571265,64673,Avg_Time_fr,0.000000e+00
571266,64677,Avg_Time_fr,0.000000e+00


In [58]:
def recommander1(final_df, user_id):

    if user_id not in list(final_df['user']):
      return("User ID not found")
    # select sub-dataset based on the module 
    #selected = [i  for i in set(final_df["item"]) if subject in i]
    sub = final_df.loc[final_df['item'].isin(['correct_Q_Math', 'correct_Q_Science', 'correct_Q_Ar', 'correct_Q_Fr'])] 
    sub = sub.dropna()
    sub['rating'] = sub['rating'] + 0.01
    # Specifiy the reader scale
    reader = Reader(rating_scale=(0, 1))
    
    # Loads Pandas dataframe
    data = Dataset.load_from_df(sub[["user", "item", "rating"]], reader)
    trainingSet = data.build_full_trainset()

    # To use item-based cosine similarity
    
    # Build the algo training 
    algo = SVD(n_epochs = 15, lr_all=0.001, reg_all=0.2, random_state=0)
    # Fit the model
    algo.fit(trainingSet)
    
    # List the recommandation for the selected user 
    recommandation = {"Subject" :[], "actual progress": [], "recommanded" : []}
    for subject in set(sub['item']) :
        prediction = algo.predict(user_id, subject)
        recommandation["Subject"].append(subject.split('_')[-1])
        recommandation["actual progress"].append(((final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == subject)])["rating"]).iloc[0])
        recommandation["recommanded"].append(prediction.est)
        #print(module , ':', prediction.est, (final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == subject)])["rating"])
    rec = pd.DataFrame(recommandation)  
    rec = rec.fillna(0)
    rec = rec.sort_values(by=['recommanded',"actual progress"])  
    return(rec)

In [71]:
recommandations = recommander1(final_df,1683)
try :
   print('Recommandation Results:')
   print(" * Top recommanded subject :",str(recommandations.iloc[-1]['Subject']))
   subject_r = str(recommandations.iloc[-1]['Subject'])
   print(" * Least recommanded subject :",str(recommandations.iloc[0]['Subject']))
except :
   pass

recommandations

Recommandation Results:
 * Top recommanded subject : Math
 * Least recommanded subject : Fr


,Subject,actual progress,recommanded
1,Fr,0.000000e+00,0.010167
3,Ar,0.000000e+00,0.010167
2,Science,0.000000e+00,0.010168
0,Math,2.460727e-07,0.010170


In [66]:
renamed = {"Math" : "رياضيات", "Fr": "Français", "Science" : "إيقاظ علمي" , "Ar" : "عربية"}

In [67]:
from os import listdir
Classes = listdir('Classes/')
Classes

['1ere.xlsx', '2eme.xlsx', '3eme.xlsx', '4eme.xlsx', '5eme.xlsx', '6eme.xlsx']

In [68]:
user_id = 1683
found = 0
for i in Classes: 
   df1 = pd.read_excel('Classes/'+i)
   df1 = reform_dataset(df1)
   if user_id in df1["** :subject (name)"]:
      found = 1
      print("User "+ str(user_id) +" in " + i.split('.')[0])
      break
if found == 0 : print('User not found!')

User 1683 in 1ere


In [69]:
def custom_recommander1(final_df, subject, user_id):
    # Build training dataframe based on modules 
    final_df = final_df.melt(id_vars="** :subject (name)")
    # Trun % progress into a rating /10 
    # final_df['value'] = final_df['value']*10 
    # Rename columns 
    final_df.columns = ["user", "item", "rating"]
    # Remove the % from the modules names
    final_df['item'] = final_df['item'].apply(lambda x: x.replace('% :', ''))

    if user_id not in list(final_df['user']):
      return("User ID not found")
    # select sub-dataset based on the module 
    selected = [i  for i in set(final_df["item"]) if subject in i]
    sub = final_df.loc[final_df['item'].isin(selected)] 
    sub = sub.dropna()

    # Specifiy the reader scale
    reader = Reader(rating_scale=(0, 10))
    
    # Loads Pandas dataframe
    data = Dataset.load_from_df(sub[["user", "item", "rating"]], reader)
    trainingSet = data.build_full_trainset()

    # To use item-based cosine similarity
    sim_options = {
        "name": "cosine",
        "user_based": False,
        "similarity_based" : True  # Compute  similarities between items
    }
    # Build the algo training 
    algo = KNNWithMeans(sim_options=sim_options, random_state=0)
    # Fit the model
    algo.fit(trainingSet)
    
    # List the recommandation for the selected user 
    recommandation = {"module" :[], "actual progress": [], "recommanded" : []}
    for module in set(sub['item']) :
        prediction = algo.predict(user_id, module)
        recommandation["module"].append(module)
        recommandation["actual progress"].append(((final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"]).iloc[0])
        recommandation["recommanded"].append(prediction.est)
        #print(module , ':', prediction.est, (final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"])
    rec = pd.DataFrame(recommandation)  
    rec = rec.fillna(0)
    rec = rec.sort_values(by=['recommanded',"actual progress"])  
    return(rec)

In [70]:
recommandations = custom_recommander1(df1, renamed[subject_r], 1683)
try :
   print('Recommandation Results:')
   print(" * Top recommanded module :",str(recommandations.iloc[-1]['module']))
   print(" * Least recommanded module :",str(recommandations.iloc[0]['module']))
except :
   pass
recommandations

Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommandation Results:
 * Top recommanded module : رياضيات (المجموعات)
 * Least recommanded module : رياضيات (التجميع)


,module,actual progress,recommanded
0,رياضيات (التجميع),0.00,0.000000
3,رياضيات (ألعاب تحدي درجة 3),0.00,0.000000
4,رياضيات (ألعاب تحدي درجة 2),0.00,0.000000
6,رياضيات (الأعداد من 0 إلى 9),0.00,0.000000
11,رياضيات (عقود),0.00,0.000000
13,رياضيات (النقود),0.00,0.000000
14,رياضيات (التوجه في الفضاء),0.00,0.000000
15,رياضيات (الجمع دون احتفاظ),0.00,0.000000
17,رياضيات (ألعاب تحدي درجة 1),0.00,0.000000
8,رياضيات (الأعداد من 10 إلى 99 ( جمع – مقارنة –...,0.00,0.032235
